In [1]:
conda install -c conda-forge ipycanvas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - ipycanvas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    freetype-2.10.4            |       h0708190_1         890 KB  conda-forge
    ipycanvas-0.9.0            |     pyhd8ed1ab_0         134 KB  conda-forge
    ipywidgets-7.6.3           |     pyhd3deb0d_0         101 KB  conda-forge
    jbig-2.1                   |    h7f98852_2003          43 KB  conda-forge
    jpeg-9d                    |       h36c2ea0_0         264 KB  conda-forge
    jupyterlab_widgets-1.0.0   |     pyhd8ed1ab_1         130 KB  conda-forge
    lcms2-2.12                 |       hddcbb42_0         443 KB  conda-forge
    lerc-2.2.

In [2]:
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType


class VM():
    def __init__(self, x,y,name="VM", size=50):
        self.x = x
        self.y = y
        self.name = name
        self.size = size
        self.Site = "UKY"
        self.coreCount = "4"
        self.ramCount = "64"
        self.diskCount = "500"
        self.IMGtype = "qcow2"
        self.IMGref = "default_centos_8"
        self.PCIs = []
        
        self.selected = False
    def draw(self):
        if self.selected:
            return "#00FF00",self.x,self.y,self.size,self.name
        else:
            return "#000000",self.x,self.y,self.size,self.name
    def setPos(self,x,y):
        self.x = x
        self.y = y
    def isSelected(self,X,Y):
        dist = pow(pow(X-self.x,2)+pow(Y-self.y,2),.5)
        if(dist<=self.size):
            self.selected=True
            return True
        else:
            self.selected=False
            return False
    def isSelectedSafe(self,X,Y):
        dist = pow(pow(X-self.x,2)+pow(Y-self.y,2),.5)
        if(dist<=self.size):
            self.selected=False
            return True
        else:
            self.selected=False
            return False
        
    def setSelected(self,state):
        self.selected=state
class VMLink:
    def __init__(self,N1,N2, size=20):
        self.Endpoints = [N1,N2]
        self.x = (N1.x + N2.x)/2
        self.y = (N1.y + N2.y)/2
        self.size = size
        self.selected=False
        self.name="link"
        self.netType="L2PTP"
        self.linkInfo = []
    def draw(self):
        color = "#FFFFFF"
        if self.selected == True:
            color = "#00FF00"
        return self.x, self.y, self.size, self.Endpoints, color
    def isSelected(self,X,Y):
        dist = pow(pow(X-self.x,2)+pow(Y-self.y,2),.5)
        if(dist<=self.size):
            self.selected=True
            return True
        else:
            self.selected=False
            return False
    def isSelectedSafe(self,X,Y):
        dist = pow(pow(X-self.x,2)+pow(Y-self.y,2),.5)
        if(dist<=self.size):
            self.selected=False
            return True
        else:
            self.selected=False
            return False
    def setPos(self,x,y):
        self.x = x
        self.y = y
    def setSelected(self,state):
        self.selected=state
    def addLink(self,VM):
        self.Endpoints.append(VM)
class FabricGUI:
    def findPCIlist(self):
        PCIs = ComponentModelType.__members__.copy()
        for i in PCIs:
            self.PCInames.append(i)
    def findNetworkTlist(self):
        NT = ServiceType.__members__.copy()
        for i in NT:
            self.networkNames.append(i)
    def deleteSVM(self,b):
        self.VMs.remove(self.selectedVM)
        self.closeVMmenu()
        self.draw()
    def deleteSLink(self,b):
        self.Links.remove(self.selectedLink)
        self.closeLinkmenu()
        self.draw()
    def loadVM(self,vm):
        self.vmName.value = vm.name
        self.Site.value = vm.Site
        self.coreCount.value = vm.coreCount
        self.ramCount.value = vm.ramCount
        self.diskCount.value = vm.diskCount
        self.IMGtype.value = vm.IMGtype
        self.IMGref = vm.IMGref
        self.loadPCIs(vm.PCIs)
    def saveVM(self,change):
        self.selectedVM.name = self.vmName.value
        self.selectedVM.Site = self.Site.value
        self.selectedVM.coreCount = self.coreCount.value
        self.selectedVM.ramCount = self.ramCount.value
        self.selectedVM.diskCount = self.diskCount.value
        self.selectedVM.IMGtype = self.IMGtype.value
        #TODO why does this value err?
        #self.selectedVM.IMGref = self.IMGref.value
        tempPCIs = []
        counter = 0
        
        for i in range(int(len(self.PCIbox.children)/2)):
            tempPCIs.append([self.PCIbox.children[counter].value,self.PCIbox.children[counter+1].value])
            counter+=2
        self.selectedVM.PCIs = tempPCIs
    def loadLink(self,link):
        self.linkName.value =link.name
        self.netType.value = link.type
        Links = []
        if link.linkInfo == []:
            for vm in link.Endpoints:
                name, pnames = self.loadNIC(vm)
                if pnames != []:
                    cur = widgets.Dropdown(options=pnames,description=name+":",disabled=False,)
                    cur.observe(self.saveLink)
                else:
                    cur = widgets.Label("VM  " + name +" has no NIC device!")
                    cur.observe(self.saveLink)
                Links.append(cur)
        else:
            for count,vm in enumerate(link.Endpoints):
                name, pnames = self.loadNIC(vm)
                if pnames != []:
                    if link.linkInfo[count] in pnames:
                        cur = widgets.Dropdown(options=pnames,value=link.linkInfo[count],description=name+":",disabled=False,)
                        cur.observe(self.saveLink)
                    else:
                        cur = widgets.Dropdown(options=pnames,description=name+":",disabled=False,)
                        cur.observe(self.saveLink)
                else:
                    cur = widgets.Label("VM  " + name +" has no NIC device!")
                Links.append(cur)
        self.Linkbox.children = Links
    def saveLink(self,change):
        self.selectedLink.name = self.linkName.value
        self.selectedLink.type = self.netType.value
        Links = []
        for i in self.Linkbox.children:
            Links.append(i.value)
        self.selectedLink.linkInfo = Links
    def loadPCIs(self,PCIs):
        pciList = []
        for i in PCIs:
            #Without this hack names don't save?
            display(i[0])
            name = widgets.Text(value=i[0],description='PCI Name:',disabled=False)
            wid = widgets.Dropdown(options=self.PCInames,value=i[1],description='PCI type:',disabled=False,)
            pciList = pciList+[name,wid]
        for i in pciList:
            i.observe(self.saveVM)
        self.PCIbox.children = pciList
    def loadNIC(self,vm):
        nicList = []
        PCIs = vm.PCIs
        for i in PCIs:
            if "NIC" in i[1]:
                nicList.append(i[0]+"-p1")
                nicList.append(i[0]+"-p2")
        return vm.name, nicList
        
        self.Linkbox.children = pciList
    def openVMmenu(self):
        for control in self.VMcontrols:
            control.layout.display = "block"
        
    def closeVMmenu(self):
        for control in self.VMcontrols:
            control.layout.display = "none"
        
        self.PCIbox.children=[]
        self.PCItemp = []
    
    def openLinkmenu(self):
        for control in self.Linkcontrols:
            control.layout.display = "block"
            
    def closeLinkmenu(self):
        for control in self.Linkcontrols:
            control.layout.display = "none"
    def addPCI(self,b):
        name = widgets.Text(value="PCIname",description='PCI Name:',disabled=False)
        wid = widgets.Dropdown(options=self.PCInames,description='PCI type:',disabled=False,)
        name.observe(self.saveVM)
        wid.observe(self.saveVM)
        self.PCItemp.append([name,wid])
        self.PCIbox.children = list(self.PCIbox.children)+[name,wid]
    def __init__(self, width=100, height =100, BG ='#584f4e'):
        self.VMs=[]
        self.Links = []
        self.PCInames = []
        self.findPCIlist()
        self.networkNames = []
        self.findNetworkTlist()
        self.PCItemp = []
        self.Held = False
        self.mouseX=0
        self.mouseY=0
        self.S1 = None
        self.S2 = None
        self.selectedVM = None
        self.selectedLink = None
        self.width=width
        self.height=height
        self.BG = BG
        self.canvas = Canvas(width=width, height=height)
        self.canvas.fill_style = BG
        self.canvas.fill_rect(0, 0, width, height)
        
        self.canvas.on_mouse_down(self.click)
        self.canvas.on_mouse_move(self.drag)
        self.canvas.on_mouse_up(self.mouseUp)
        
        self.Options = widgets.RadioButtons(
            options=['Create', 'Move', 'Link'],
            value='Create',
            layout={'display': 'flex'}, # If the items' names are long
            description='Edit Option:',
            disabled=False)
        self.compileButton = widgets.Button(description="Compile to Code")
        self.compileButton.on_click(self.CompileCode)
        
        display(self.Options)
        display(self.compileButton)
        display(self.canvas)
        
        #Edit node menu
        self.vmName =  widgets.Text(value="VM",description='VM Name:',disabled=False)
        self.Site = widgets.Dropdown(options=['UKY', 'RENC', 'LBNL', 'STAR'],value='UKY',description='Site:',disabled=False,)
        self.coreCount = widgets.Text(value="4",description='Core:',disabled=False)
        self.ramCount = widgets.Text(value="64",description='Ram:',disabled=False)
        self.diskCount = widgets.Text(value="500",description='Disk:',disabled=False)
        self.IMGtype =widgets.Dropdown(options=['qcow2'],value='qcow2',description='Image Type:',disabled=False,)
        self.IMGref = widgets.Dropdown(options=['default_centos_8'],value='default_centos_8',description='Image Reference',disabled=False,)
        self.PCIbox = widgets.Box()
        self.deleteButton = widgets.Button(description="Delete VM", button_style="danger")
        self.deleteButton.on_click(self.deleteSVM)
        self.addPCIButton =  widgets.Button(description="Add PCI")
        self.addPCIButton.on_click(self.addPCI)
        self.VMcontrols = [self.vmName,self.Site,self.coreCount,self.ramCount,self.diskCount,self.IMGtype,self.IMGref,self.PCIbox,self.addPCIButton,self.deleteButton]
        for control in self.VMcontrols:
            control.observe(self.saveVM)
            display(control)
        self.closeVMmenu()
        
        #Edit link menu
        self.linkName =  widgets.Text(value="Name",description='Network Name:',disabled=False)
        self.netType = widgets.Dropdown(options=self.networkNames,description='Network Type:',disabled=False,)
        self.Linkbox = widgets.Box()
        self.deleteButtonLink = widgets.Button(description="Delete Link", button_style="danger")
        self.deleteButtonLink.on_click(self.deleteSLink)
        self.Linkcontrols = [self.linkName,self.netType,self.Linkbox,self.deleteButtonLink]
        for control in self.Linkcontrols:
            control.observe(self.saveLink)
            display(control)
        self.closeLinkmenu()
    def createLink(self,N1,N2):
        newLink = VMLink(N1,N2)
        self.Links.append(newLink)
    def Create(self,x,y):
        
        if [vm for vm in self.VMs if vm.selected]:
            [vm.setSelected(False) for vm in self.VMs if vm.selected]
            self.draw()
            return False
        
        checkPos = list(set([currentCheck.isSelectedSafe(x,y) for currentCheck in self.VMs]))
        
        if len(checkPos) == 1:
            
            if checkPos[0]==False:
                
                new = VM(x,y)
                new.setSelected(False)
                self.VMs.append(new)
                self.draw()
            else:
                self.draw()
                
        elif len(checkPos) == 0:
            
            new = VM(x,y)
            new.setSelected(False)
            self.VMs.append(new)
            self.draw()
        else:
            self.draw()
    def Move(self,x,y):
        
        if [vm for vm in self.VMs if vm.selected]:
            [vm.setSelected(False) for vm in self.VMs if vm.selected]
            self.selectedVM = None
            self.selectedLink = None
            self.closeVMmenu()
            self.closeLinkmenu()
            self.draw()
            return False
        if [link for link in self.Links if link.selected]:
            [link.setSelected(False) for link in self.Links if link.selected]
            self.selectedVM = None
            self.selectedLink = None
            self.closeLinkmenu()
            self.closeVMmenu()
            self.draw()
            return False
        for curr in self.VMs:
            if curr.isSelected(x,y):
                self.selectedVM = curr
                self.loadVM(self.selectedVM)
                self.openVMmenu()
                return
        for curr in self.Links:
            if curr.isSelected(x,y):
                self.selectedLink = curr
                self.loadLink(self.selectedLink)
                self.openLinkmenu()
                return
            
    def clickLink(self,x,y):
        if self.S1 == None:
            for currentCheck in self.VMs:
                if currentCheck.isSelected(x,y):
                    self.S1 = currentCheck
                    break
        else:
            for currentCheck in self.VMs:
                if currentCheck.isSelected(x,y):
                    self.S2 = currentCheck
                    break
            if self.S1 != self.S2 and self.S2 != None:
                self.createLink(self.S1,self.S2)
            else:
                for currentCheck in self.Links:
                    if currentCheck.isSelected(x,y):
                        currentCheck.addLink(self.S1)
            if [vm for vm in self.VMs if vm.selected]:
                [vm.setSelected(False) for vm in self.VMs if vm.selected]
                self.draw()
            if [link for link in self.Links if link.selected]:
                [link.setSelected(False) for link in self.Links if link.selected]
                self.draw()
                return False
            self.S1=None
            self.S2=None
    def click(self,x,y):
        
        if (self.Options.value == "Create"):
            self.Held = True
            self.Create(x,y)
        elif (self.Options.value == "Move"):
            self.Held = True
            self.Move(x,y)
        elif (self.Options.value == "Link"):
            self.clickLink(x,y)
                
    
                
    def mouseUp(self,x,y):
        self.Held = False
    def drag(self,x,y):
        self.mouseX=x
        self.mouseY=y
        if self.Held:
            if [vm for vm in self.VMs if vm.selected]:
                #self.canvas.fill_style = "#0000FF"
                #self.canvas.fill_circle(x,y,5)
                with hold_canvas(self.canvas):
                    [vm for vm in self.VMs if vm.selected][-1].setPos(x,y)
            else:
                if [link for link in self.Links if link.selected]:
                    with hold_canvas(self.canvas):
                        [link for link in self.Links if link.selected][-1].setPos(x,y)
                    
        
        self.draw()

    def clearCanvas(self):
        self.canvas.clear()
        self.canvas.fill_style=self.BG
        self.canvas.fill_rect(0, 0, self.width, self.height)
    def drawInfo(self,info):
        
        self.canvas.text_align="start"
        self.canvas.fill_style = "#000000"
        self.canvas.fill_text(info, 0, 50)
    def draw(self):
        with hold_canvas(self.canvas):
            #Refresh
            self.clearCanvas()
            
            #Draw links
            for link in self.Links:
                mx,my,size,nodes, color = link.draw()
                for node in nodes:
                    self.canvas.stroke_line(mx,my,node.x,node.y)
                    
                self.canvas.fill_style = color
                self.canvas.fill_circle(mx,my,size)
            
            #Draw indicator line
            if(self.S1 is not None):
                self.canvas.stroke_line(self.S1.x,self.S1.y,self.mouseX,self.mouseY)
            
            #Draw VMs
            for vm in self.VMs:
                color,x,y,size,name = vm.draw()
                self.canvas.fill_style = color
                self.canvas.fill_circle(x,y,size)
                self.canvas.fill_style = "#FFFFFF"
                self.canvas.font = '32px arial'
                self.canvas.text_align="center"
                self.canvas.fill_text(name, x, y)
    def CompileCode(self,b):
        self.errorCheck()
        
        code = "t = ExperimentTopology()\n\n"
        
        for count, vm in enumerate(self.VMs):
            countString = str(count+1)
            code+= "node"+countString +" = t.add_node(name='"+vm.name+"', site='"+vm.Site+"')\n\n"
            code+="cap"+countString+" = Capacities()\n"
            code+="cap"+countString+".set_fields(core="+vm.coreCount+", ram="+vm.ramCount+", disk="+vm.diskCount+")\n\n"
        display(code)
    def errorCheck(self):
        #TODO
        return
        
  


In [3]:
%%html
<style>
    .widget-radio-box {
        flex-direction: row !important;     
    }
    .widget-radio-box label{
        margin:0px !important;
        width: 70px !important;
    }
</style>
              

In [5]:
MainGui = FabricGUI(800,500)

MainGui.draw()

RadioButtons(description='Edit Option:', layout=Layout(display='flex'), options=('Create', 'Move', 'Link'), va…

Button(description='Compile to Code', style=ButtonStyle())

Canvas(width=800)

Text(value='VM', description='VM Name:')

Dropdown(description='Site:', options=('UKY', 'RENC', 'LBNL', 'STAR'), value='UKY')

...

Button(button_style='danger', description='Delete VM', style=ButtonStyle())

Text(value='Name', description='Network Name:')

Dropdown(description='Network Type:', options=('P4', 'MPLS', 'OVS', 'L2Path', 'L2STS', 'L2PTP', 'L2Multisite',…

Box()

Button(button_style='danger', description='Delete Link', style=ButtonStyle())